# Setup environment

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Clone `CreateDebateScraper` library from github
!git clone https://github.com/utkarsh512/CreateDebateScraper.git
%cd CreateDebateScraper/src/nested/

In [ ]:
from   copy                     import deepcopy
from   itertools                import accumulate
import json
from   matplotlib               import pyplot as plt
import networkx as nx
import nltk
import numpy as np
import pandas as pd
import pickle
import re
from   scipy                    import stats
import textwrap
from   thread                   import Comment, Thread
from   tqdm                     import tqdm
nltk.download('punkt') # For tokenizers
import matplotlib
from   nltk.tokenize            import TweetTokenizer
from   pprint                   import pprint
matplotlib.rcParams.update({'font.size': 18})

# Helper functions

In [ ]:
tknz = TweetTokenizer()

def clean_text(text):
    """
    Preprocessing text
    """
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www\S+", "", text)
    text = re.sub("-", " ", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\u2018", "X", text) 
    text = re.sub("\u2019", "X", text) 
    text = re.sub("\'", "X", text) 
    wordTokens_ = tknz.tokenize(text)
    wordTokens = list()
    for x in wordTokens_:
        x = ''.join([v for v in x if v.isalnum() or v == ' '])
        if len(x) > 0 and x != 'X':
            x = x.replace('X', '\'')
            wordTokens.append(x)
    return wordTokens

In [ ]:
SLUR_WORDS = {
  "jews": [
    "jews",
    "oven dodger",
    "nazi",
    "dirty jew",
    "holocaust",
    "kikesucker",
    "hook nose",
    "kike"
  ],
  "homosexual": [
    "faggots usually",
    "fucking queer",
    "the biggest faggot",
    "dyke",
    "you fucking faggot",
    "hate faggots",
    "queer",
    "homosexual",
    "the faggots",
    "faggot",
    "faggots usually have",
    "gay",
    "faggots",
    "dykey",
    "ugly dyke",
    "faggots like you",
    "you a fag",
    "lesbian",
    "homo",
    "is a faggot",
    "like a faggot",
    "dykes",
    "faggots like",
    "faggot if you ever"
  ],
  "women": [
    "ugly dyke",
    "woman terrorist",
    "nigress",
    "bitch",
    "slut",
    "women",
    "sheeboon",
    "negress",
    "mud shark",
    "women threat",
    "you a lame bitch",
    "your a cunt",
    "white bitch",
    "niggeress",
    "hoe",
    "dykes",
    "niggress",
    "sheboon",
    "feminazi"
  ],
  "blacks": [
    "pavement ape",
    "the niggers",
    "negress",
    "porch monkey",
    "that nigger",
    "this nigger",
    "sheboon",
    "all niggers",
    "eurafrica",
    "shut up nigger",
    "picaninny",
    "african attack",
    "spearchucker",
    "how many niggers",
    "nigger",
    "africa",
    "niggers are in my",
    "dindu nuffin",
    "stupid nigger",
    "moolie",
    "niggers",
    "bluegum",
    "nigger ass",
    "you niggers",
    "fucking nigger",
    "nigger music",
    "niggress",
    "you a nigger",
    "many niggers are",
    "nigress",
    "blacks",
    "teenaper",
    "sheeboon",
    "dumb nigger",
    "niggeress",
    "pickaninny",
    "nigga"
  ],
  "muslim": [
    "muslim immigrant",
    "islam",
    "mudslime",
    "mooslem",
    "muslim refugee",
    "musslime",
    "shitlam",
    "muslim invasion",
    "moslime",
    "mooslamic",
    "muzzie",
    "allah akbar",
    "mooslime",
    "musloid",
    "mudslimes",
    "muslim",
    "muslimes",
    "moslum",
    "mussie",
    "muzrat",
    "muslim countries",
    "muzzy",
    "moslim",
    "jihadi",
    "muslim country",
    "moslem",
    "muzzrat",
    "mooslim"
  ],
  "arabs": [
    "towel head",
    "goatfucker",
    "arabs",
    "goathumper",
    "raghead",
    "rag head",
    "goathumping",
    "towelhead",
    "camel jockey",
    "sandnigger",
    "camel fucker",
    "sand nigger"
  ],
  "generic": [
    "to rape",
    "raped and",
    "shithole country",
    "get raped",
    "raped",
    "is a fucking",
    "shit skin",
    "raped by",
    "hate you",
    "fake empowerment",
    "abusive women",
    "fuck you too",
    "violence",
    "wit a lame nigga",
    "they all look",
    "alllivesmatter",
    "shithole countries",
    "fucking hate",
    "trailer trash",
    "kill all",
    "terrorist threat",
    "harassment",
    "kill yourself",
    "shitskin",
    "okay to be white",
    "fucking hate you"
  ],
  "white": [
    "full of white",
    "white trash",
    "white devil",
    "white",
    "are all white",
    "white boy",
    "white ass",
    "white bitch",
    "hillbilly",
    "whigger",
    "white christian",
    "white person",
    "all white",
    "white nigger",
    "redneck",
    "white honky",
    "wigger",
    "them white"
  ],
  "economy": [
    "ghetto"
  ],
  "immigrant": [
    "illegal immigrants",
    "immigrant not welcome",
    "immigrant terror",
    "mexcrement",
    "go back to where you come from",
    "muslim refugee",
    "illegal aliens",
    "refugee",
    "protect from immigrants",
    "negro",
    "refugees",
    "immigrant",
    "refugee invasion",
    "go back to where they come from",
    "refugees impact",
    "bring ebola",
    "immigrants",
    "illegal alien",
    "immigrant invasion",
    "bring disease"
  ],
  "mental": [
    "retard",
    "mongoloid",
    "retarded"
  ],
  "asians": [
    "asians",
    "ching chong",
    "chinaman"
  ]
}

# Loading CreateDebate dataset

In [ ]:
comments = dict()

# Topical forums on CreateDebate. We have scraped comments for all of the
# following forurm.
categories = ['business', 'comedy', 'entertainment', 'health', 'law', 'nsfw',
              'politics2', 'religion', 'science', 'shopping', 'sports',
              'technology', 'travel', 'world']

# However, we will be analyzing comments from selected forum only!
# These forum have at least 10k comments each.
categories_selected = ['politics2', 'religion', 'world', 
                       'science', 'law', 'technology']

for x in categories_selected:
    comments[x] = list()

In [ ]:
# Loading comments from select forums

for cat in tqdm(categories_selected):
    fp = open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/threads.log', 'rb')

    # Get all the `Thread` objects pickled while scraping.
    threads = list()
    try:
        while True:
            e = pickle.load(fp)
            threads.append(e)
    except EOFError:
        fp.close()

    # While classifying CreateDebate comments, we used comments as per author mode.
    # Hence, using the same mode to attach classification score with the comments.
    # 
    # score < 0.5 -> ad hominem comment
    #       > 0.5 -> non ad hominem comment
    authors = dict()
    for thread in threads:
        for k, v in thread.comments.items():
            try:
                authors[v.author].append(v)
            except:
                authors[v.author] = list()
                authors[v.author].append(v)

    ctr = 0
    # Load the classification score of the comments.
    with open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/comments_with_score.log', 'rb') as fp:
        cws = pickle.load(fp)
    # Attach classification score with the comments.
    for author in authors.keys():
        for i in range(len(authors[author])):
            comment = authors[author][i]
            foo = deepcopy(comment.__dict__)
            foo['tag'] = cat
            foo['score'] = cws[ctr][0]
            foo['validation'] = cws[ctr][1][0]
            comments[cat].append(foo)
            ctr += 1

# Analyzing dataset for slur words across ad hominem score

In [ ]:
ah_score = dict()
slur_count = dict()

# addressing order: 
#   ah_score: 
#       key: category -> author
#       value: list of score of each comment written
#
#   slur_count:
#       key: category -> slur_group -> author 
#       value: list of slur count of each comment written

In [ ]:
for cat in categories_selected:
    ah_score[cat] = dict()
    slur_count[cat] = dict()

    for slur_group in SLUR_WORDS.keys():
        slur_count[cat][slur_group] = dict()

    for comment in tqdm(comments[cat]): 
        author = comment['author']
        if author not in ah_score[cat]:
            ah_score[cat][author] = list()
            for slur_group in SLUR_WORDS.keys():
                slur_count[cat][slur_group][author] = list()
        ah_score[cat][author].append(1 - comment['score'])
        cleaned_comment_text = clean_text(comment['body'])
        for slur_group, slur_words in SLUR_WORDS.items():
            current_count = sum([cleaned_comment_text.count(slur_word) for slur_word in slur_words])
            slur_count[cat][slur_group][author].append(current_count)

In [ ]:
# Median ah score per category per author
#   key: category -> author
#   value: median ah score
ah_score_median = dict()

for category, author_data in ah_score.items():
    ah_score_median[category] = dict()
    for author, ah_scores in author_data.items():
        ah_score_median[category][author] = np.median(ah_scores)

In [ ]:
HIGH_AH_THRESHOLD = 0.8
MODERATE_AH_THRESHOLD = 0.2

high_ah_group = dict()
moderate_ah_group = dict()

# key: category -> slur_group
# value: list of slur_count

In [ ]:
for category in categories_selected:
    high_ah_group[category] = dict()
    moderate_ah_group[category] = dict()

    for slur_group in SLUR_WORDS.keys():
        high_ah_group[category][slur_group] = list()
        moderate_ah_group[category][slur_group] = list()

    for author, median_ah_score in ah_score_median[category].items():
        if median_ah_score < MODERATE_AH_THRESHOLD:
            continue
        elif median_ah_score > HIGH_AH_THRESHOLD:
            for slur_group in SLUR_WORDS.keys():
                high_ah_group[category][slur_group].append(np.sum(slur_count[category][slur_group][author]) / len(ah_score[category][author]))
        else:
            for slur_group in SLUR_WORDS.keys():
                moderate_ah_group[category][slur_group].append(np.sum(slur_count[category][slur_group][author]) / len(ah_score[category][author]))

In [ ]:
# How many users considered for each case

report = ' ' * 20 + ', '
for category in categories_selected:
    report += f'{category:>20}, '
for slur_group in SLUR_WORDS.keys():
    report += '\n' + f'{slur_group:20}, '
    for category in categories_selected:
        report += f'{len(high_ah_group[category][slur_group]):>10}/{len(moderate_ah_group[category][slur_group]):<9}, '
print(report)

# Plotting utilities

In [ ]:
def plot_category(category, func=np.average):
    x = list(SLUR_WORDS.keys())
    y1 = []
    y2 = []

    for slur_group in x:
        y1.append(func(high_ah_group[category][slur_group]))
        y2.append(func(moderate_ah_group[category][slur_group]))
    ticks = np.arange(len(x))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10, 5))
    subplot1 = ax.bar(ticks - width/2, y1, width, label='High', tick_label=x)
    subplot2 = ax.bar(ticks + width/2, y2, width, label='Moderate', tick_label=x)
    ax.set_ylabel('Scores')
    ax.set_title(f'Slur word distribution for \'{category}\' forum')
    ax.set_xticks(ticks)
    ax.set_xticklabels(x, rotation=45, ha='right')
    ax.legend()
    plt.show()

def plot_slurs(slur_group, func=np.average):
    x = categories_selected
    y1 = []
    y2 = []

    for category in x:
        y1.append(func(high_ah_group[category][slur_group]))
        y2.append(func(moderate_ah_group[category][slur_group]))
    ticks = np.arange(len(x))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10, 5))
    subplot1 = ax.bar(ticks - width/2, y1, width, label='High', tick_label=x)
    subplot2 = ax.bar(ticks + width/2, y2, width, label='Moderate', tick_label=x)
    ax.set_ylabel('Scores')
    ax.set_title(f'Slur word distribution for \'{slur_group}\' slurs')
    ax.set_xticks(ticks)
    ax.set_xticklabels(x, rotation=45, ha='right')
    ax.legend()
    plt.show()

# Results

## For given category

In [ ]:
plot_category('politics2')

In [ ]:
plot_category('religion')

In [ ]:
plot_category('world')

In [ ]:
plot_category('science')

In [ ]:
plot_category('law')

In [ ]:
plot_category('technology')

## For given slur group

In [ ]:
plot_slurs('jews')

In [ ]:
plot_slurs('homosexual')

In [ ]:
plot_slurs('women')

In [ ]:
plot_slurs('blacks')

In [ ]:
plot_slurs('muslim')

In [ ]:
plot_slurs('arabs')

In [ ]:
plot_slurs('generic')

In [ ]:
plot_slurs('white')

In [ ]:
plot_slurs('economy')

In [ ]:
plot_slurs('immigrant')

In [ ]:
plot_slurs('mental')

In [ ]:
plot_slurs('asians')